1. Import all the Required Liberaries

In [99]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os
import sys

2. Load the Dataset --> CSV File

In [100]:
DB_FAISS_PATH = "vectorstore/db_faiss"

In [101]:
loader = CSVLoader(file_path="D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\air_water_data.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
print(data)

[Document(page_content='City: New York City\nCountry: "United States of America"\nAirQuality: 46.81603774\nWaterPollution: 49.5049505', metadata={'source': 'D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\air_water_data.csv', 'row': 0}), Document(page_content='City: Washington, D.C.\nCountry: "United States of America"\nAirQuality: 66.12903226\nWaterPollution: 49.10714286', metadata={'source': 'D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\air_water_data.csv', 'row': 1}), Document(page_content='City: San Francisco\nCountry: "United States of America"\nAirQuality: 60.51401869\nWaterPollution: 43', metadata={'source': 'D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\air_water_data.csv', 'row': 2}), Document(page_content='City: Berlin\nCountry: "Germany"\nAirQuality: 62.36413043\nWaterPollution: 28.61271676', metadata={'source': 'D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\air_water_data.csv', 'row': 3}), Docume

3. Split the Text into Chunks

In [102]:
# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [103]:
print(len(text_chunks))

17


4. Embedding := Use the Sentence Transformer Embedding from Hugging Face Liberary

In [104]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [105]:
#from langchain.embeddings import OpenAIEmbeddings

#embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("OpenAI_API_Key"))

5. FAISS Vector Space is Used to Store the Embeddings

In [106]:
# COnverting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

docsearch.save_local(DB_FAISS_PATH)

6. OpenAI LLM Model Gpt is Used for Data Analysis

In [107]:
load_dotenv()
openai_api_key = os.getenv("OpenAI_API_Key")
os.environ["OPENAI_API_KEY"] = openai_api_key
llm = OpenAI(temperature=0)

In [108]:
#from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain import LLMMathChain

In [109]:
#from langchain.llms import CTransformers

In [110]:
#llm = CTransformers(model="D:\\Data_analytics_new\\Generative_AI\\Langchain\\Langchain-Work\\model\\llama-2-7b-chat.ggmlv3.q4_0.bin",
 #                   model_type="llama",
  #                  max_new_tokens=512,
   #                 temperature=0.1)

In [111]:
#LLM_Math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
#math_tool = Tool.from_function(func = LLM_Math_chain.run)
#tools = load_tools(["llm-math"],llm=llm)

In [112]:
#agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [113]:
#agent.run("columns name in dataset")

In [114]:
chain = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

In [119]:
while True:
    chat_history = []
    query = input(f"Input Prompt: ")
    if query == 'exit':
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = chain({"question":query, "chat_history":chat_history})
    print("Response: ", result['answer'])

Response:   The airquality of New York City is 46.81603774.
Exiting


SystemExit: 

c:\Users\Oliver\anaconda4\envs\llms\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [55]:
#chat_history = []
#query = "How many rows are there in the dataset"
#response = chain({"question": query,"chat_history":chat_history})
#print(response['answer'])

 4
